In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}
url = "https://hvl.instructure.com/api/v1/accounts/1/users"

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


# Finne/slette gamle brukarar
Vi må ha ei rydderutine for å finne gamle eksterne brukarar. Dette er dei som er registret direkte i Canvas, og som ikkje har vore innlogga på "ei stund".

Først henter eg ut alle brukarar i Canvas:
```

In [2]:
dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            dataliste.append([element['id'], element['name'], element['sis_user_id'], element['created_at']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'sis_user_id', 'created_at'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


https://hvl.instructure.com/api/v1/accounts/1/users?page=2&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=3&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=4&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=5&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=6&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=7&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=8&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=9&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=10&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=11&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=12&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=13&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=14&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=15

Så ser eg litt på kor mange det er og dei fem første:

In [5]:
alledata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82765 entries, 0 to 82764
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           82765 non-null  int64 
 1   name         82765 non-null  object
 2   sis_user_id  82176 non-null  object
 3   created_at   82765 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [6]:
alledata.head()

,id,name,sis_user_id,created_at
0,44998,Mattevakten,hvl200622-p1-akv,2020-06-22T14:18:44+02:00
1,1703,Terje Rudi (admin),hvl230817-p3-dp,2017-12-16T00:21:52+01:00
2,108712,Gunhild Skåsheim (student),HVL240919-P1-JEO,2024-09-19T22:43:10+02:00
3,96153,Terje Rudi (student),fs:203:265856,2023-08-17T11:48:55+02:00
4,54230,Head of International Relations-faculty of Soc...,hvl-200917-p5-jbru,2020-09-17T10:25:23+02:00


## Finne eksterne
Det er nokre eksterne brukarar som ikkje har `sis_user_id`, så eg plukker ut dei først i ei eiga liste (og tar dei vekk frå hovudlista):


In [43]:
minus_sis = alledata[alledata['sis_user_id'].isnull()]
resten = alledata[~alledata['sis_user_id'].isnull()]
minus_sis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 589 entries, 5 to 82466
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           589 non-null    int64 
 1   name         589 non-null    object
 2   sis_user_id  0 non-null      object
 3   created_at   589 non-null    object
dtypes: int64(1), object(3)
memory usage: 23.0+ KB


Så kan eg finne dei i `resten` som *ikkje* starter med "fs:203":

In [44]:
eksterne_temp = resten[~(resten['sis_user_id'].str.startswith("fs:203") | resten['sis_user_id'].str.startswith("FS:203") | resten['sis_user_id'].str.startswith("Fs:203"))]
eksterne_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11305 entries, 0 to 82764
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           11305 non-null  int64 
 1   name         11305 non-null  object
 2   sis_user_id  11305 non-null  object
 3   created_at   11305 non-null  object
dtypes: int64(1), object(3)
memory usage: 441.6+ KB


Så er det igjen nokon av desse som kjem frå andre høgskular (via Trust account):

In [45]:
trust = eksterne_temp[eksterne_temp['sis_user_id'].str.startswith("fs:")] 
resten = eksterne_temp[~eksterne_temp['sis_user_id'].str.startswith("fs:")]
trust.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5265 entries, 10 to 82745
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5265 non-null   int64 
 1   name         5265 non-null   object
 2   sis_user_id  5265 non-null   object
 3   created_at   5265 non-null   object
dtypes: int64(1), object(3)
memory usage: 205.7+ KB


Det er ganske mange, faktisk halvparten av alle. No skal `resten` vere alle som har sis_id som ikkje starter på "fs:". Men eg kan jo sjekke det:

In [46]:
print(f"Det er {len(resten)} personar i lista.")
ikkje_hvl = resten[~(resten['sis_user_id'].str.contains("hvl") | resten['sis_user_id'].str.contains("HVL"))]
print(f"Det er {len(ikkje_hvl)} personar i lista som ikkje starter med hvl eller HVL.")

Det er 6040 personar i lista.
Det er 239 personar i lista som ikkje starter med hvl eller HVL.


Eg lagrer data slik at eg slepp å lese dei om att frå API:

In [66]:
resten.to_csv("eksterne_brukarar_1_250221.csv", index=False)

In [67]:
minus_sis.to_csv("eksterne_brukarar_2_250221.csv", index=False)

## Finne sist pålogging
No kjem det vanskeleg arbeidet: å finne "siste pålogging" for kvar av dei 6040 brukarane som har "hvl"-id og dei 589 som ikkje har sis_id i det heile. 

Eg kan prøve med endepunktet `/api/v1/users/:user_id/logins`: 

In [55]:
user_id = resten.iloc[-1]['id']
print(user_id)
url = f"https://hvl.instructure.com/api/v1/users/{user_id}/logins"
resultat = requests.get(url, headers=headers)
resultat.json()

73707


[{'id': 74096,
  'user_id': 73707,
  'account_id': 1,
  'workflow_state': 'active',
  'unique_id': 'Paula.Zwo@t-online.de',
  'created_at': '2021-08-18T05:53:56Z',
  'sis_user_id': 'hvl210816-p26-akv',
  'integration_id': None,
  'authentication_provider_id': 1,
  'declared_user_type': None,
  'authentication_provider_type': 'canvas'}]

Den ga meg fint lite, i alle fall ikkje det som står i dokumentasjonen. 

Kanskje eg skal prøve å lese ffrå CD2; i tabellen `users` er det eit felt som heiter "last_logged_out". [Det hjalp ikkje; ikkje mykje informasjon der.] 

Neste steg er tabellen `enrollments`; her får eg feltet `last_activity_at` som i følge dokumentasjonen er 

> The last activity time of the user for the enrollment. This time stamp updates when a user navigates through the course using the Course Navigation menu, replies to a discussion or submits an assignment or quiz, and when they access course files and pages (does not include mobile interactions or group participation).

Det eg får ut er siste gang ein brukar var inne i eit gitt emne. Så eg må sjå på *alle* slike oppføringar for ein brukar, og så plukke ut den yngste. NB! Eg har henta data med skriptet "les_CD2_tabell.ipynb", men eg har berre lagra `user_id` og `course_id`. Dokumentasjonen tilrår å sjekke `course_section_id` og `role_id` også, for en brukar kan jo ha vore logga på som student i eit emne og litt seinare som tilsett (eller studentassistent) i emnet. Men eg trur risikoen for dette er liten (det er jo eksterne faglærarar og kursdeltakarar dette).

In [73]:
data = pd.read_csv('brukarar_last_activity_at.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302226 entries, 0 to 302225
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   value.user_id           302226 non-null  int64 
 1   value.course_id         302226 non-null  int64 
 2   value.last_activity_at  223576 non-null  object
dtypes: int64(2), object(1)
memory usage: 6.9+ MB


Eg hentar ut alle `user_id` i tabellen:

In [65]:
brukarar = data['value.user_id'].unique()
print(f"Det er {len(brukarar)} unike brukarar eg må sjekke.")

Det er 45641 unike brukarar eg må sjekke.


Ikkje alle desse er nødvendig å sjekke; eg skal jo berre ha dei som er eksterne. Desse hentar eg frå filane "eksterne_brukarar_1_250221.csv" og "eksterne_brukarar_2_250221.csv":

In [72]:
eksterne_1 = pd.read_csv("eksterne_brukarar_1_250221.csv")
eksterne_2 = pd.read_csv("eksterne_brukarar_2_250221.csv")
eksterne_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           6040 non-null   int64 
 1   name         6040 non-null   object
 2   sis_user_id  6040 non-null   object
 3   created_at   6040 non-null   object
dtypes: int64(1), object(3)
memory usage: 188.9+ KB


Så går eg gjennom kvar enkelt av brukarane i desse tofilene, og sjekker om `id` er i `value.user_id` i `brukarar`. Er dei det så finn eg alle oppføringar, og plukker ut den siste:

In [83]:
for uid in eksterne_2['id']:
    if len(data[data['value.user_id']==uid]) > 0:
        print(f"{uid}: {data[data['value.user_id']==uid]}")

812:         value.user_id  value.course_id value.last_activity_at
125927            812              576                    NaN
96676:         value.user_id  value.course_id    value.last_activity_at
289170          96676            30265  2025-02-19T07:57:51.077Z
816:         value.user_id  value.course_id value.last_activity_at
125919            816              576                    NaN
684:         value.user_id  value.course_id value.last_activity_at
125473            684              575                    NaN
1506:        value.user_id  value.course_id value.last_activity_at
63923           1506              575                    NaN
829:         value.user_id  value.course_id value.last_activity_at
125999            829              576                    NaN
833:         value.user_id  value.course_id value.last_activity_at
126005            833              576                    NaN
837:         value.user_id  value.course_id value.last_activity_at
125989            837  

In [81]:
brukarar

array([ 67355,  47509,  94554, ..., 111246, 111247, 111248], dtype=int64)